# TD7 - AURELIEN POUXVIEL NLP

### Loading data for Keras

In [12]:
! pip install transformers datasets
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split

## Dataset Load

In [13]:
dataset = load_dataset("glue", "cola")
dataset = dataset["train"]

## Subset because way too long

In [14]:
subset_size = 1000
dataset = dataset.select(range(subset_size))

In [15]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

## Tokenizer

In [16]:
# Initialiser le tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenized_train_data = tokenizer(train_dataset["sentence"], return_tensors="tf", padding=True, truncation=True)
tokenized_train_data = dict(tokenized_train_data)
labels_train = np.array(train_dataset["label"])

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [17]:
tokenized_test_data = tokenizer(test_dataset["sentence"], return_tensors="tf", padding=True, truncation=True)
tokenized_test_data = dict(tokenized_test_data)
labels_test = np.array(test_dataset["label"])

## Model Training and fine tuning

In [18]:
## Parralelisation
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
    loss_fn = SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=Adam(learning_rate=3e-5), loss=loss_fn, metrics=['accuracy'])

# Fine-tuning
model.fit(tokenized_train_data, labels_train, epochs=1, batch_size=4)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


200/200 [==============================] - 658s 3s/step - loss: 0.6441 - accuracy: 0.6375


## As expected, the fine tunning is long
## SO thats why i reduced as i could the model effiency by reducing epochs, length of the data etc

In [19]:
# Evaluation sur le subset de test
eval_results = model.evaluate(tokenized_test_data, labels_test)
print("Loss on test set:", eval_results[0])
print("Accuracy on test set:", eval_results[1])

7/7 [==============================] - 30s 3s/step - loss: 0.5984 - accuracy: 0.6650
Loss on test set: 0.5984253287315369
Accuracy on test set: 0.6650000214576721
